In [1]:
from torchvision import datasets, transforms
import math
import torch
import torch.nn.functional as nnf
import torch.nn as nn
import os
import time
import torch.optim as optim 

#### Model

In [2]:
# get a pretrained network for transfer learning
import torchvision.models as models
densenet = models.densenet161(pretrained=True)

class transfer_music_classifer(nn.Module):
  def __init__(self):
    super(transfer_music_classifer, self).__init__()
    # increase the mono-channel spectrogram to 3 channels for use
    # in pretrained densenet. Padding of 1 to preserve image size
    self.channelUpscale = nn.Conv2d(in_channels = 3, out_channels = 3, kernel_size = 3, padding = 1)
    
    self.featureExtract = densenet.features
    
    self.classifier = nn.Sequential(
      # mary modify this, add more layers, add dropout, etc
      nn.Linear(2208, 780),
      nn.ReLU(),
      nn.Dropout(0.2),
      nn.Linear(780, 240),
      nn.ReLU(),
      nn.Dropout(0.2),
      nn.Linear(240, 8),
      nn.ReLU()
    )

  def forward(self, spectrogram):
    upscaled = self.channelUpscale(spectrogram)
    
    features = self.featureExtract(upscaled)

    ######
    # This code is normally built into densenet, but since we are splitting
    # the features and classifier it must be put here
    # Taken from here: 
    # https://pytorch.org/vision/stable/_modules/torchvision/models/densenet.html

    # inplace = True means it modifies the input instead of allocating memory
    # for an ouput. Saves on memory
    out = nnf.relu(features, inplace=True)

    # adaptive pool decides the stride and kernel size automatically to ensure
    # the output has shape (x,x,1,1) regardless of input
    out = nnf.adaptive_avg_pool2d(out, (1, 1))

    # reshape output to be (1, x)
    out = torch.flatten(out, 1)
    ######    

    out = self.classifier(out)
    return out

#### Data loading

In [3]:
torch.manual_seed(1000)

# root dir for dataset !!! CHANGE TO OWN !!!
#root_path = r'D:\aps360\spectrograms'
root_path = r'C:\Users\sjd30\Docnosync\spectrograms'
# root_path = r'C:\Users\Matthew\Downloads\data'

mytransforms = transforms.Compose([
  transforms.Resize([224,224]),
#   transforms.Grayscale(num_output_channels=1),
  transforms.ToTensor(),
  # densenet expects these transformations
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# load the dataset & labels from the given folder. Convert them to tensors
dataset = datasets.ImageFolder(root_path, transform=mytransforms)
print("Total num images:", len(dataset))

num_train_samples = math.floor(len(dataset)*0.7)
num_val_samples = math.floor(len(dataset)*0.15)
num_test_samples = math.floor(len(dataset)*0.15)

# add any extra images to the test set to avoid imperfect splitting
num_test_samples += len(dataset) - (num_train_samples + num_val_samples + num_test_samples)
print("number of images in train,val,test,total: ", num_train_samples, num_val_samples, num_test_samples, num_train_samples + num_val_samples + num_test_samples)

# uses pytorch method to randomly divide the loaded dataset
train_data, val_data, test_data = torch.utils.data.random_split(dataset, [num_train_samples, num_val_samples, num_test_samples])

# load the train dataset in on an alternate core (using a worker)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, 
                                          num_workers=0, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=64, 
                                          num_workers=0, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, 
                                         num_workers=0, shuffle=True)

Total num images: 230530
number of images in train,val,test,total:  161371 34579 34580 230530


#### Functions

In [10]:
# model is the network
# train_data and val_data are pytorch datasets
# batch_size, learning_rate, and num_epochs are hyperparameters
# save_dir defines the directory in which models will be saved
# net_name makes it easy to find the saved model in drive
# weight_decay is a parameter used in adam optimizer to 

def train_net(model, train_data, val_data, batch_size=32, learning_rate=1e-3, num_epochs=5, save=False, save_dir=r'C:\Users\sjd30\Downloads', weight_decay=1e-5, net_name='default'):

  # CE Loss for multi-class
  criterion = nn.CrossEntropyLoss()

  # adam optimizer for rapid training
  # the pretrained features of densenet are trained at 1/3 the learning rate
  # for fine tuning purposes.
  optimizer = optim.Adam([
                {'params': model.channelUpscale.parameters() },
                {'params': model.featureExtract.parameters(), 'lr': learning_rate / 3.0},
                {'params': model.classifier.parameters() }
            ], lr=learning_rate, weight_decay = weight_decay)

  trainacc, valacc, iterations, lossrecords = [], [], [], []

  # create the dataloaders from the passed data
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                          num_workers=0, shuffle=True)
  val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                            num_workers=0, shuffle=True)
  batch_num = 0
  start=time.time()
  for epoch in range(num_epochs):
    mini_b=0
    mini_batch_correct = 0
    mini_batch_total = 0
    #loops through each minibatch
    for spectrograms, labels in iter(train_loader):
      #Checks for GPU availability. Uses GPU if available.
      if torch.cuda.is_available():
        spectrograms = spectrograms.cuda()
        labels = labels.cuda()

      # standard training loop for pytroch NN.
      # gets outputs for the batch, computes losses,
      # uses backpropagation to compute the gradient
      # adjusts the weights, and cleans up the gradient
      out = model(spectrograms)
      loss = criterion(out, labels)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad(set_to_none=True)
    
      pred = out.max(1, keepdim=True)[1]
      mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
      mini_batch_total = spectrograms.shape[0]

      lossrecords.append(float(loss) / batch_size) # convert to float
      trainacc.append((mini_batch_correct / mini_batch_total))
      #valacc.append(eval_model(model, val_loader))
      iterations.append(batch_num)

      # each iteration give basic progress info
      print('Batch Number{0}  Total Time (s): {1}  Progress: {2}  Trainacc: {3}'.format(batch_num, time.time()-start, (100 * batch_num * batch_size) / (num_epochs * len(train_loader) * batch_size), trainacc[-1]))
      # progress to next batch
      mini_b += 1
      batch_num += 1

    # saves the file with a descriptive name, including a timestamp to help
    # prevent overwriting files in case the net_name was not changed
    # also, since the model is large (100mb), dont save it unless it has
    # improved since the last epoch. -2 fixes a possible issue where the last
    # minibatch has skewed accuracy due to a different size
    # https://piazza.com/class/ko91w5f8es3619?cid=70
    if save:
      if valacc[-2] > valacc[int(-(batch_num / (epoch + 1) - 1))]:
        print('saving model')
        save_file = os.path.join(save_dir,'name-{0}_epoch-{1}_bs-{2}_lr-{3}_time-{4}'.format(net_name, epoch, batch_size, learning_rate, datetime.datetime.now().strftime("%H-%M-%S")))
        torch.save(model.state_dict(), save_file)

    # each epoch, give more detailed accuracy information
    print('Epoch number: {0}  Train acc: {1}  Val acc: {2}   Loss{3}\nTotal Time (s): {4}   Time per Epoch (s): {5}  \n\n'.format(
        epoch+1, trainacc[-1], valacc[-1], lossrecords[-1], time.time() - start, (time.time() - start) / (1 + epoch) ))

In [11]:
def eval_model(model, loader_to_evaluate):
  correct = 0
  total = 0

  for spectrograms, labels in loader_to_evaluate:
    # use GPU
    if torch.cuda.is_available():
      spectrograms = spectrograms.cuda()
      labels = labels.cuda()

    # the output from model(spectrograms) is the probabilities of each genre
    # as the rows, and the classes as the columns. We find the max
    # probability class (dim = 1) and that returns a 2d tensor with only
    # columns. To make this a 1d tensor, we take the dim 1 (like squeezing)
    classPrediction = model(spectrograms).max(1, keepdim=True)[1]
    # adds up all of the  times that the label equal the prediction. Taken
    # from tut3a notebook
    correct += classPrediction.eq(labels.view_as(classPrediction)).sum().item()
    total += spectrograms.shape[0]

  return correct / total

In [12]:
def set_parameter_requires_grad(model):
    for param in model.featureExtract.parameters():
        param.requires_grad = False

#### Training

In [ ]:
model = transfer_music_classifer()

# Train on GPU
if torch.cuda.is_available():
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU, consider using a GPU runtime to speed up training')
    
# set_parameter_requires_grad(model)
train_net(model, train_data, val_data, batch_size=16, num_epochs=50, learning_rate = 1e-3, save=True)

Using GPU
Batch Number0  Total Time (s): 0.38133764266967773  Progress: 0.0  Trainacc: 0.0625
Batch Number1  Total Time (s): 0.7338359355926514  Progress: 0.000198294665873488  Trainacc: 0.1875
Batch Number2  Total Time (s): 1.0933506488800049  Progress: 0.000396589331746976  Trainacc: 0.3125
Batch Number3  Total Time (s): 1.456624984741211  Progress: 0.000594883997620464  Trainacc: 0.125
Batch Number4  Total Time (s): 1.8180146217346191  Progress: 0.000793178663493952  Trainacc: 0.0
Batch Number5  Total Time (s): 2.194345474243164  Progress: 0.00099147332936744  Trainacc: 0.125
Batch Number6  Total Time (s): 2.5639777183532715  Progress: 0.001189767995240928  Trainacc: 0.0
Batch Number7  Total Time (s): 2.9161765575408936  Progress: 0.001388062661114416  Trainacc: 0.3125
Batch Number8  Total Time (s): 3.2900493144989014  Progress: 0.001586357326987904  Trainacc: 0.0625
Batch Number9  Total Time (s): 3.661015748977661  Progress: 0.001784651992861392  Trainacc: 0.0625
Batch Number10  To

Batch Number83  Total Time (s): 30.82699680328369  Progress: 0.016458457267499504  Trainacc: 0.125
Batch Number84  Total Time (s): 31.194246768951416  Progress: 0.01665675193337299  Trainacc: 0.0625
Batch Number85  Total Time (s): 31.54733967781067  Progress: 0.01685504659924648  Trainacc: 0.0625
Batch Number86  Total Time (s): 31.895821571350098  Progress: 0.017053341265119967  Trainacc: 0.0
Batch Number87  Total Time (s): 32.245166301727295  Progress: 0.017251635930993457  Trainacc: 0.125
Batch Number88  Total Time (s): 32.59146428108215  Progress: 0.017449930596866944  Trainacc: 0.3125
Batch Number89  Total Time (s): 32.94900369644165  Progress: 0.017648225262740434  Trainacc: 0.0625
Batch Number90  Total Time (s): 33.305328607559204  Progress: 0.01784651992861392  Trainacc: 0.0625
Batch Number91  Total Time (s): 33.66580104827881  Progress: 0.01804481459448741  Trainacc: 0.125
Batch Number92  Total Time (s): 34.017343282699585  Progress: 0.018243109260360896  Trainacc: 0.0625
Batch

Batch Number166  Total Time (s): 62.023192405700684  Progress: 0.03291691453499901  Trainacc: 0.0625
Batch Number167  Total Time (s): 62.37599587440491  Progress: 0.0331152092008725  Trainacc: 0.1875
Batch Number168  Total Time (s): 62.73348307609558  Progress: 0.03331350386674598  Trainacc: 0.0625
Batch Number169  Total Time (s): 63.08232593536377  Progress: 0.03351179853261947  Trainacc: 0.125
Batch Number170  Total Time (s): 63.439342737197876  Progress: 0.03371009319849296  Trainacc: 0.125
Batch Number171  Total Time (s): 63.796496868133545  Progress: 0.03390838786436645  Trainacc: 0.25
Batch Number172  Total Time (s): 64.14706373214722  Progress: 0.034106682530239935  Trainacc: 0.0
Batch Number173  Total Time (s): 64.49437236785889  Progress: 0.034304977196113424  Trainacc: 0.125
Batch Number174  Total Time (s): 64.8825433254242  Progress: 0.034503271861986914  Trainacc: 0.125
Batch Number175  Total Time (s): 65.2527871131897  Progress: 0.0347015665278604  Trainacc: 0.125
Batch Nu

Batch Number249  Total Time (s): 95.4076235294342  Progress: 0.04937537180249851  Trainacc: 0.25
Batch Number250  Total Time (s): 95.76150250434875  Progress: 0.049573666468372  Trainacc: 0.375
Batch Number251  Total Time (s): 96.11882710456848  Progress: 0.049771961134245486  Trainacc: 0.125
Batch Number252  Total Time (s): 96.4616174697876  Progress: 0.049970255800118976  Trainacc: 0.0625
Batch Number253  Total Time (s): 96.82751798629761  Progress: 0.050168550465992466  Trainacc: 0.1875
Batch Number254  Total Time (s): 97.19461488723755  Progress: 0.050366845131865956  Trainacc: 0.25
Batch Number255  Total Time (s): 97.54129528999329  Progress: 0.05056513979773944  Trainacc: 0.125
Batch Number256  Total Time (s): 97.8829927444458  Progress: 0.05076343446361293  Trainacc: 0.0625
Batch Number257  Total Time (s): 98.22798895835876  Progress: 0.05096172912948642  Trainacc: 0.0
Batch Number258  Total Time (s): 98.57650303840637  Progress: 0.0511600237953599  Trainacc: 0.125
Batch Number2

Batch Number332  Total Time (s): 124.64380359649658  Progress: 0.06583382906999802  Trainacc: 0.125
Batch Number333  Total Time (s): 124.98862338066101  Progress: 0.0660321237358715  Trainacc: 0.25
Batch Number334  Total Time (s): 125.33762168884277  Progress: 0.066230418401745  Trainacc: 0.0625
Batch Number335  Total Time (s): 125.68832588195801  Progress: 0.06642871306761848  Trainacc: 0.1875
Batch Number336  Total Time (s): 126.04144716262817  Progress: 0.06662700773349196  Trainacc: 0.0625
Batch Number337  Total Time (s): 126.38606691360474  Progress: 0.06682530239936546  Trainacc: 0.0
Batch Number338  Total Time (s): 126.73174214363098  Progress: 0.06702359706523894  Trainacc: 0.125
Batch Number339  Total Time (s): 127.07863259315491  Progress: 0.06722189173111243  Trainacc: 0.0
Batch Number340  Total Time (s): 127.43449568748474  Progress: 0.06742018639698592  Trainacc: 0.1875
Batch Number341  Total Time (s): 127.77899074554443  Progress: 0.0676184810628594  Trainacc: 0.0625
Batc

...

Batch Number6132  Total Time (s): 2099.53049325943  Progress: 1.2159428911362284  Trainacc: 0.0625
Batch Number6133  Total Time (s): 2099.8555569648743  Progress: 1.2161411858021018  Trainacc: 0.125
Batch Number6134  Total Time (s): 2100.182494163513  Progress: 1.2163394804679755  Trainacc: 0.125
Batch Number6135  Total Time (s): 2100.524985551834  Progress: 1.2165377751338489  Trainacc: 0.1875
Batch Number6136  Total Time (s): 2100.8549389839172  Progress: 1.2167360697997225  Trainacc: 0.125
Batch Number6137  Total Time (s): 2101.191655397415  Progress: 1.2169343644655959  Trainacc: 0.1875
Batch Number6138  Total Time (s): 2101.5259590148926  Progress: 1.2171326591314693  Trainacc: 0.0
Batch Number6139  Total Time (s): 2101.8539905548096  Progress: 1.217330953797343  Trainacc: 0.1875
Batch Number6140  Total Time (s): 2102.1829392910004  Progress: 1.2175292484632163  Trainacc: 0.1875
Batch Number6141  Total Time (s): 2102.5164923667908  Progress: 1.21772754312909  Trainacc: 0.0625
Batc

Batch Number6215  Total Time (s): 2127.071944475174  Progress: 1.232401348403728  Trainacc: 0.125
Batch Number6216  Total Time (s): 2127.410993337631  Progress: 1.2325996430696013  Trainacc: 0.0625
Batch Number6217  Total Time (s): 2127.7449967861176  Progress: 1.232797937735475  Trainacc: 0.0625
Batch Number6218  Total Time (s): 2128.072523832321  Progress: 1.2329962324013484  Trainacc: 0.125
Batch Number6219  Total Time (s): 2128.405140399933  Progress: 1.233194527067222  Trainacc: 0.0625
Batch Number6220  Total Time (s): 2128.7409060001373  Progress: 1.2333928217330954  Trainacc: 0.1875
Batch Number6221  Total Time (s): 2129.068815469742  Progress: 1.2335911163989688  Trainacc: 0.0625
Batch Number6222  Total Time (s): 2129.4039902687073  Progress: 1.2337894110648424  Trainacc: 0.125
Batch Number6223  Total Time (s): 2129.733946323395  Progress: 1.2339877057307158  Trainacc: 0.0625
Batch Number6224  Total Time (s): 2130.0659506320953  Progress: 1.2341860003965894  Trainacc: 0.1875
Ba

Batch Number6298  Total Time (s): 2154.6489984989166  Progress: 1.2488598056712275  Trainacc: 0.0
Batch Number6299  Total Time (s): 2154.986271381378  Progress: 1.2490581003371009  Trainacc: 0.125
Batch Number6300  Total Time (s): 2155.3189928531647  Progress: 1.2492563950029745  Trainacc: 0.125
Batch Number6301  Total Time (s): 2155.648950099945  Progress: 1.2494546896688479  Trainacc: 0.25
Batch Number6302  Total Time (s): 2155.98389339447  Progress: 1.2496529843347215  Trainacc: 0.0
Batch Number6303  Total Time (s): 2156.311953306198  Progress: 1.249851279000595  Trainacc: 0.125
Batch Number6304  Total Time (s): 2156.651943206787  Progress: 1.2500495736664683  Trainacc: 0.1875
Batch Number6305  Total Time (s): 2156.9809477329254  Progress: 1.250247868332342  Trainacc: 0.0
Batch Number6306  Total Time (s): 2157.3079521656036  Progress: 1.2504461629982153  Trainacc: 0.0625
Batch Number6307  Total Time (s): 2157.6499462127686  Progress: 1.250644457664089  Trainacc: 0.3125
Batch Number6

Batch Number6381  Total Time (s): 2182.294958114624  Progress: 1.265318262938727  Trainacc: 0.1875
Batch Number6382  Total Time (s): 2182.624771595001  Progress: 1.2655165576046004  Trainacc: 0.125
Batch Number6383  Total Time (s): 2182.959958791733  Progress: 1.265714852270474  Trainacc: 0.1875
Batch Number6384  Total Time (s): 2183.2849926948547  Progress: 1.2659131469363474  Trainacc: 0.0625
Batch Number6385  Total Time (s): 2183.613992214203  Progress: 1.266111441602221  Trainacc: 0.0625
Batch Number6386  Total Time (s): 2183.95449924469  Progress: 1.2663097362680944  Trainacc: 0.125
Batch Number6387  Total Time (s): 2184.290949344635  Progress: 1.2665080309339678  Trainacc: 0.0625
Batch Number6388  Total Time (s): 2184.618948698044  Progress: 1.2667063255998414  Trainacc: 0.0
Batch Number6389  Total Time (s): 2184.944991350174  Progress: 1.2669046202657148  Trainacc: 0.1875
Batch Number6390  Total Time (s): 2185.280976295471  Progress: 1.2671029149315884  Trainacc: 0.125
Batch Num

Batch Number6464  Total Time (s): 2210.284900903702  Progress: 1.2817767202062265  Trainacc: 0.1875
Batch Number6465  Total Time (s): 2210.6199905872345  Progress: 1.2819750148720999  Trainacc: 0.0625
Batch Number6466  Total Time (s): 2210.9439713954926  Progress: 1.2821733095379735  Trainacc: 0.0625
Batch Number6467  Total Time (s): 2211.2839918136597  Progress: 1.2823716042038469  Trainacc: 0.25
Batch Number6468  Total Time (s): 2211.6109552383423  Progress: 1.2825698988697205  Trainacc: 0.125
Batch Number6469  Total Time (s): 2211.9514966011047  Progress: 1.282768193535594  Trainacc: 0.1875
Batch Number6470  Total Time (s): 2212.2779591083527  Progress: 1.2829664882014673  Trainacc: 0.125
Batch Number6471  Total Time (s): 2212.6059560775757  Progress: 1.283164782867341  Trainacc: 0.125
Batch Number6472  Total Time (s): 2212.950998067856  Progress: 1.2833630775332143  Trainacc: 0.0625
Batch Number6473  Total Time (s): 2213.2894999980927  Progress: 1.283561372199088  Trainacc: 0.0
Bat

The stuff below here is probably bad or otherwise unecesary


In [ ]:
# this is for testing purposes only
import glob
from PIL import Image
import matplotlib.pyplot as plt

images=glob.glob("/content/149244013_c529578289.jpg")
for image in images:
  img = Image.open(image).convert('L')
  trans = transforms.ToPILImage()
  trans1 = transforms.ToTensor()
  tens = trans(trans1(img))
  plt.imshow(tens)

  model = transfer_music_classifer()
  spectrogram = trans1(img).unsqueeze(0)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay = 1e-5)

  print( model(spectrogram) )
  optimizer.zero_grad()
  out = model(spectrogram)
  loss = criterion(out, torch.empty(1, dtype=torch.long).random_(5))
  loss.backward()
  print(optimizer.step())


In [ ]:
# dont include in final report because i literally stole this from here https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model
# this just counts parameters
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

net = transfer_music_classifer()
count_parameters(net)

+--------------------------------------------------------+------------+
|                        Modules                         | Parameters |
+--------------------------------------------------------+------------+
|                featureExtract.0.weight                 |     27     |
|                 featureExtract.0.bias                  |     3      |
|             featureExtract.1.conv0.weight              |   14112    |
|             featureExtract.1.norm0.weight              |     96     |
|              featureExtract.1.norm0.bias               |     96     |
| featureExtract.1.denseblock1.denselayer1.norm1.weight  |     96     |
|  featureExtract.1.denseblock1.denselayer1.norm1.bias   |     96     |
| featureExtract.1.denseblock1.denselayer1.conv1.weight  |   18432    |
| featureExtract.1.denseblock1.denselayer1.norm2.weight  |    192     |
|  featureExtract.1.denseblock1.denselayer1.norm2.bias   |    192     |
| featureExtract.1.denseblock1.denselayer1.conv2.weight  |   829

26489702